pre-trained) 기존 모델 내의 weight들이 fitting되어있는 경우

In [1]:
import torch
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [5]:
from torchvision import datasets, transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # RGB 평균, 평균, 표준편차.
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder('./data/hymenoptera_data', data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 8, num_workers = 0, shuffle = True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [6]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate (train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

In [7]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

- 이미지와 레이블을 test_loader에서 추출해 DEVICE에 할당.
- 그 결과값을 cross_entropy로 계산.
- 그 배치 안에 있는 레이블에 대한 모든 결과값을 sum해서 test_loss에 더해줌.
- output 값 중 가장 높은 값을 클래스로 예측.
- 그 예측값과 정답이 동일한 경우에만 correct를 증가.

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# model = models.resnet18(pretrained = False).cuda()
model = models.resnet18(pretrained = False).cpu()
optimizer = optim.Adam(model.parameters(), lr = 0.0001) # pretrained 된 값에는 학습률을 적게.
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders['train'], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders['val'])
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/794 (0%)]	Loss: 6.859541
Train Epoch: 1 [80/794 (10%)]	Loss: 1.424673
Train Epoch: 1 [160/794 (20%)]	Loss: 1.740338
Train Epoch: 1 [240/794 (30%)]	Loss: 1.307625
Train Epoch: 1 [320/794 (40%)]	Loss: 1.189715
Train Epoch: 1 [400/794 (50%)]	Loss: 1.116353
Train Epoch: 1 [480/794 (60%)]	Loss: 1.151788
Train Epoch: 1 [560/794 (70%)]	Loss: 0.956495
Train Epoch: 1 [640/794 (80%)]	Loss: 1.152220
Train Epoch: 1 [720/794 (90%)]	Loss: 1.509945
[1] Test Loss: 1.1252, accuracy: 47.98%

Train Epoch: 2 [0/794 (0%)]	Loss: 1.022793
Train Epoch: 2 [80/794 (10%)]	Loss: 1.322210
Train Epoch: 2 [160/794 (20%)]	Loss: 1.234704
Train Epoch: 2 [240/794 (30%)]	Loss: 1.117810
Train Epoch: 2 [320/794 (40%)]	Loss: 1.363701
Train Epoch: 2 [400/794 (50%)]	Loss: 1.186017
Train Epoch: 2 [480/794 (60%)]	Loss: 1.450052
Train Epoch: 2 [560/794 (70%)]	Loss: 1.084153
Train Epoch: 2 [640/794 (80%)]	Loss: 1.247823
Train Epoch: 2 [720/794 (90%)]	Loss: 0.905574
[2] Test Loss: 1.6918, accuracy: 27.46%

Train 

In [ ]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # imagenet의 클래스는 1000개로 설정. output node를 2개로 조정.

if USE_CUDA:
    model = model.cuda() # GPU로 변경.
    
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders['train'], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders['val'])
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

In [ ]:
model = models.resnet18(pretrained = True)

for param in model.parameters():
    param.requires_grad = False # 가져온 weight들이 update되지 않음.
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr = 0.0001)
EPOCHS = 10

for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders['train'], optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, dataloaders['val'])
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

데이터가 너무 적은 경우에는 gradient를 freezing 시키는 것이 좋지 않을 수도 있음.